# Dependency Injection in FastAPI 🔌

This notebook covers everything you need to know about Dependency Injection in FastAPI!

**What you'll learn:**
1. **Understanding Dependencies** - What is dependency injection and how it works
2. **Advanced Dependency Patterns** - Class-based dependencies, sub-dependencies, overrides
3. **Common Use Cases** - Database sessions, authentication, validation, rate limiting, logging

Dependency Injection is one of FastAPI's most powerful features - it helps you write clean, testable, and maintainable code! 🚀

## Part 5.1: Understanding Dependencies

### What is Dependency Injection?

**Dependency Injection (DI)** is a design pattern where:
- Objects receive their dependencies from external sources (rather than creating them internally)
- FastAPI automatically resolves and injects dependencies into your route functions
- This makes your code more modular, testable, and maintainable

**Benefits:**
- ✅ Separation of concerns
- ✅ Easier testing (can mock dependencies)
- ✅ Code reusability
- ✅ Better organization

In [1]:
# Basic Example: Function-based Dependency

from fastapi import FastAPI, Depends

app = FastAPI()

# Simple dependency function
def get_db_connection():
    """Simulates a database connection"""
    print("🔌 Creating database connection...")
    return {"connection": "active", "db": "postgresql"}

# Using dependency in a route
@app.get("/items/")
def get_items(db: dict = Depends(get_db_connection)):
    """
    The 'db' parameter will automatically receive the result of get_db_connection()
    FastAPI calls get_db_connection() and passes its return value to db
    """
    return {
        "items": ["item1", "item2", "item3"],
        "db_status": db
    }

print("✅ Basic dependency example created!")
print("💡 When you call /items/, FastAPI will:")
print("   1. Call get_db_connection()")
print("   2. Pass its return value to the 'db' parameter")
print("   3. Execute your route function")

✅ Basic dependency example created!
💡 When you call /items/, FastAPI will:
   1. Call get_db_connection()
   2. Pass its return value to the 'db' parameter
   3. Execute your route function


### Creating Dependency Functions

Dependencies can be:
- **Functions** (most common)
- **Classes** (for more complex scenarios)
- **Async functions** (for async operations)

Let's see different ways to create dependencies:

In [ ]:
# Example 1: Simple Function Dependency

from fastapi import FastAPI, Depends, Query
from typing import Optional

app = FastAPI()

def get_pagination_params(
    skip: int = Query(0, ge=0, description="Number of items to skip"),
    limit: int = Query(10, ge=1, le=100, description="Number of items to return")
):
    """Dependency that extracts and validates pagination parameters"""
    return {"skip": skip, "limit": limit}

@app.get("/products/")
def list_products(pagination: dict = Depends(get_pagination_params)):
    """List products with pagination"""
    return {
        "pagination": pagination,
        "products": [f"Product {i}" for i in range(pagination["skip"], pagination["skip"] + pagination["limit"])]
    }

print("✅ Pagination dependency created!")

TypeError: 'Depends' object is not subscriptable

In [ ]:
# Example 2: Async Dependency Function

import asyncio
from fastapi import FastAPI, Depends

app = FastAPI()

async def get_async_data():
    """Async dependency - useful for async database operations"""
    print("⏳ Fetching data asynchronously...")
    await asyncio.sleep(0.1)  # Simulate async operation
    return {"data": "async_data", "timestamp": "2024-01-01"}

@app.get("/async-data/")
async def get_data(data: dict = Depends(get_async_data)):
    """Route using async dependency"""
    return {"result": data}

print("✅ Async dependency example created!")

In [ ]:
# Example 3: Dependency with Parameters

from fastapi import FastAPI, Depends, Header
from typing import Optional

app = FastAPI()

def get_user_agent(user_agent: Optional[str] = Header(None)):
    """Dependency that extracts User-Agent header"""
    return user_agent or "Unknown"

@app.get("/user-info/")
def get_user_info(ua: str = Depends(get_user_agent)):
    """Route that uses User-Agent from header"""
    return {
        "user_agent": ua,
        "message": f"Request from: {ua}"
    }

print("✅ Dependency with header parameter created!")

### Using Dependencies in Routes

There are multiple ways to use dependencies:

1. **As function parameters** (most common)
2. **As route decorator** (applies to all routes)
3. **As router dependencies** (applies to all routes in a router)

In [ ]:
# Method 1: Dependency as Function Parameter (Most Common)

from fastapi import FastAPI, Depends

app = FastAPI()

def get_settings():
    return {"app_name": "MyApp", "version": "1.0.0"}

@app.get("/info/")
def get_info(settings: dict = Depends(get_settings)):
    """Dependency injected as function parameter"""
    return {"info": settings}

print("✅ Method 1: Parameter injection")

In [ ]:
# Method 2: Route-level Dependencies (Applies to all methods)

from fastapi import FastAPI, Depends, APIRouter, Header, HTTPException

app = FastAPI()

def verify_api_key(api_key: str = Header(..., alias="X-API-Key")):
    """Verify API key from header"""
    if api_key != "secret-key-123":
        raise HTTPException(status_code=403, detail="Invalid API key")
    return {"api_key": api_key, "verified": True}

# Apply dependency to all HTTP methods for this route
@app.api_route("/protected/", methods=["GET", "POST"], dependencies=[Depends(verify_api_key)])
def protected_route():
    """This route requires API key verification"""
    return {"message": "Access granted", "data": "sensitive information"}

print("✅ Method 2: Route-level dependencies")

In [ ]:
# Method 3: Router-level Dependencies

from fastapi import FastAPI, Depends, APIRouter, Header
from fastapi.exceptions import HTTPException

app = FastAPI()

def check_admin(role: str = Header(..., alias="X-User-Role")):
    """Check if user is admin"""
    if role != "admin":
        raise HTTPException(status_code=403, detail="Admin access required")
    return {"role": role, "access": "admin"}

# Create router with dependencies
admin_router = APIRouter(
    prefix="/admin",
    dependencies=[Depends(check_admin)]  # Applied to ALL routes in this router
)

@admin_router.get("/users/")
def list_users():
    """All routes in admin_router require admin role"""
    return {"users": ["user1", "user2"]}

@admin_router.get("/settings/")
def get_settings():
    """This route also requires admin role"""
    return {"settings": "admin_settings"}

app.include_router(admin_router)

print("✅ Method 3: Router-level dependencies")

### Dependency Scopes and Lifetime

FastAPI supports different dependency scopes:

1. **Request Scope** (default) - Dependency is created for each request
2. **Singleton Scope** - Dependency is created once and reused
3. **Application Scope** - Dependency lives for the entire application lifetime

Let's explore these:

In [ ]:
# Example: Request Scope (Default Behavior)

from fastapi import FastAPI, Depends
import time

app = FastAPI()

# Counter to track how many times dependency is called
call_count = 0

def get_request_id():
    """This dependency is called ONCE per request"""
    global call_count
    call_count += 1
    request_id = f"req-{call_count}-{int(time.time() * 1000)}"
    print(f"🆔 Creating request ID: {request_id}")
    return request_id

@app.get("/request-scope/")
def get_data(request_id: str = Depends(get_request_id)):
    """Each request gets a new request_id"""
    return {
        "request_id": request_id,
        "message": "This dependency is created fresh for each request"
    }

print("✅ Request scope example (default behavior)")
print("💡 Each request will get a new request_id")

In [ ]:
# Example: Caching Dependencies (Singleton-like behavior)

from fastapi import FastAPI, Depends
from functools import lru_cache

app = FastAPI()

@lru_cache()  # Cache the result - only called once
def get_app_config():
    """This dependency is cached - only called once"""
    print("⚙️ Loading application configuration (only once!)")
    return {
        "app_name": "MyApp",
        "version": "1.0.0",
        "environment": "production",
        "database_url": "postgresql://localhost/db"
    }

@app.get("/config/")
def get_config(config: dict = Depends(get_app_config)):
    """Config is loaded once and cached"""
    return {"config": config}

@app.get("/config/check/")
def check_config(config: dict = Depends(get_app_config)):
    """Same cached config is reused"""
    return {"config": config, "cached": True}

print("✅ Cached dependency example")
print("💡 get_app_config() is only called once, then cached")

## Part 5.2: Advanced Dependency Patterns

Now let's explore advanced patterns for more complex scenarios!

### Class-based Dependencies

Use classes when you need to maintain state or have multiple methods.

In [ ]:
# Example: Class-based Dependency

from fastapi import FastAPI, Depends, Header
from typing import Optional

app = FastAPI()

class UserService:
    """Class-based dependency for user operations"""
    
    def __init__(self, user_id: Optional[str] = Header(None, alias="X-User-ID")):
        self.user_id = user_id or "anonymous"
        self.loaded = False
    
    def load_user(self):
        """Simulate loading user data"""
        if not self.loaded:
            print(f"👤 Loading user: {self.user_id}")
            self.loaded = True
        return {
            "user_id": self.user_id,
            "name": f"User {self.user_id}",
            "email": f"{self.user_id}@example.com"
        }
    
    def get_permissions(self):
        """Get user permissions"""
        return ["read", "write"] if self.user_id != "anonymous" else ["read"]

# FastAPI will instantiate UserService for each request
@app.get("/user/")
def get_user(user_service: UserService = Depends(UserService)):
    """Route using class-based dependency"""
    user = user_service.load_user()
    permissions = user_service.get_permissions()
    return {
        "user": user,
        "permissions": permissions
    }

print("✅ Class-based dependency example")
print("💡 UserService is instantiated for each request")

### Sub-dependencies (Dependencies of Dependencies)

Dependencies can depend on other dependencies, creating a dependency chain!

In [ ]:
# Example: Sub-dependencies (Dependencies of Dependencies)

from fastapi import FastAPI, Depends, Header, HTTPException
from typing import Optional

app = FastAPI()

# Level 1: Base dependency
def get_api_key(api_key: Optional[str] = Header(None, alias="X-API-Key")):
    """Extract API key from header"""
    if not api_key:
        raise HTTPException(status_code=401, detail="API key required")
    return api_key

# Level 2: Depends on get_api_key
def verify_api_key(api_key: str = Depends(get_api_key)):
    """Verify the API key (depends on get_api_key)"""
    valid_keys = ["key-123", "key-456", "key-789"]
    if api_key not in valid_keys:
        raise HTTPException(status_code=403, detail="Invalid API key")
    return {"api_key": api_key, "verified": True}

# Level 3: Depends on verify_api_key
def get_user_permissions(auth: dict = Depends(verify_api_key)):
    """Get user permissions (depends on verify_api_key)"""
    # In real app, you'd look up permissions based on API key
    return {
        "permissions": ["read", "write", "delete"],
        "api_key": auth["api_key"]
    }

@app.get("/protected-data/")
def get_protected_data(permissions: dict = Depends(get_user_permissions)):
    """
    This route depends on get_user_permissions, which depends on verify_api_key,
    which depends on get_api_key.
    
    Dependency chain: get_api_key → verify_api_key → get_user_permissions → route
    """
    return {
        "data": "sensitive information",
        "permissions": permissions
    }

print("✅ Sub-dependency example")
print("💡 Dependency chain: get_api_key → verify_api_key → get_user_permissions")

In [ ]:
# Example: Complex Sub-dependency Chain

from fastapi import FastAPI, Depends, Header, HTTPException
from typing import Optional

app = FastAPI()

# Database connection (simulated)
class Database:
    def __init__(self):
        self.connected = False
    
    def connect(self):
        if not self.connected:
            print("🔌 Connecting to database...")
            self.connected = True
        return self
    
    def query(self, sql: str):
        return {"result": f"Executed: {sql}"}

# Dependency 1: Get database connection
def get_database():
    """Create database connection"""
    db = Database()
    return db.connect()

# Dependency 2: Get user (depends on database)
def get_current_user(db: Database = Depends(get_database)):
    """Get current user from database"""
    # Simulate query
    return {"user_id": "123", "username": "john_doe", "role": "user"}

# Dependency 3: Check permissions (depends on user)
def check_permission(permission: str, user: dict = Depends(get_current_user)):
    """Check if user has permission"""
    def permission_checker():
        user_role = user.get("role")
        if permission == "admin" and user_role != "admin":
            raise HTTPException(status_code=403, detail="Admin permission required")
        return {"permission": permission, "granted": True, "user": user}
    return permission_checker

@app.get("/admin-panel/")
def admin_panel(
    user: dict = Depends(get_current_user),
    permission: dict = Depends(check_permission("admin"))
):
    """Route with complex dependency chain"""
    return {
        "message": "Admin panel access granted",
        "user": user,
        "permission": permission
    }

print("✅ Complex sub-dependency chain example")

### Dependency Overrides (for Testing)

One of the most powerful features: you can override dependencies for testing!

In [ ]:
# Example: Dependency Overrides for Testing

from fastapi import FastAPI, Depends
from fastapi.testclient import TestClient

app = FastAPI()

# Production dependency
def get_database():
    """Real database connection"""
    print("🔌 Connecting to REAL database...")
    return {"db": "production", "connection": "real"}

@app.get("/data/")
def get_data(db: dict = Depends(get_database)):
    """Route using database dependency"""
    return {"data": "sensitive data", "db": db}

# Test dependency (mock)
def get_test_database():
    """Mock database for testing"""
    print("🧪 Using TEST database...")
    return {"db": "test", "connection": "mock"}

# Override the dependency
app.dependency_overrides[get_database] = get_test_database

# Now test the endpoint
client = TestClient(app)
response = client.get("/data/")
print("✅ Dependency override example")
print(f"Response: {response.json()}")
print("💡 The route now uses get_test_database instead of get_database!")

# Clean up (important!)
app.dependency_overrides.clear()
print("\n🧹 Overrides cleared")

In [ ]:
# Real Testing Example with Dependency Overrides

from fastapi import FastAPI, Depends, HTTPException
from fastapi.testclient import TestClient

app = FastAPI()

# Production: Real authentication
def get_current_user(token: str = Header(..., alias="Authorization")):
    """Real authentication - would verify JWT token"""
    if token != "Bearer real-token":
        raise HTTPException(status_code=401, detail="Invalid token")
    return {"user_id": "123", "username": "real_user"}

@app.get("/profile/")
def get_profile(user: dict = Depends(get_current_user)):
    """Get user profile"""
    return {"profile": user}

# Test: Mock authentication
def get_test_user():
    """Mock user for testing"""
    return {"user_id": "test-123", "username": "test_user"}

# Test the endpoint
def test_profile_endpoint():
    """Test endpoint with mocked dependency"""
    # Override dependency
    app.dependency_overrides[get_current_user] = get_test_user
    
    client = TestClient(app)
    response = client.get("/profile/")
    
    assert response.status_code == 200
    assert response.json()["profile"]["username"] == "test_user"
    
    # Clean up
    app.dependency_overrides.clear()
    print("✅ Test passed! Dependency override worked correctly")

# Run the test
test_profile_endpoint()

### Caching Dependencies

Use `@lru_cache` or `functools.cache` to cache expensive dependencies.

In [ ]:
# Example: Caching Expensive Dependencies

from fastapi import FastAPI, Depends
from functools import lru_cache
import time

app = FastAPI()

# Expensive operation (e.g., loading config file, connecting to external service)
@lru_cache(maxsize=1)  # Cache only the latest result
def load_expensive_config():
    """Expensive operation that should be cached"""
    print("⏳ Loading expensive configuration (this takes time!)...")
    time.sleep(0.5)  # Simulate expensive operation
    return {
        "api_keys": {"service1": "key1", "service2": "key2"},
        "settings": {"timeout": 30, "retries": 3},
        "loaded_at": time.time()
    }

@app.get("/config/")
def get_config(config: dict = Depends(load_expensive_config)):
    """First call loads config, subsequent calls use cache"""
    return {"config": config}

@app.get("/config/again/")
def get_config_again(config: dict = Depends(load_expensive_config)):
    """This call uses cached config - much faster!"""
    return {"config": config}

print("✅ Caching example")
print("💡 First call: loads config (slow)")
print("💡 Second call: uses cache (fast)")

## Part 5.3: Common Use Cases

Now let's see how dependencies are used in real-world scenarios!

### 1. Database Session Management

One of the most common uses: managing database connections and sessions.

In [ ]:
# Example: Database Session Management

from fastapi import FastAPI, Depends
from contextlib import contextmanager
from typing import Generator

app = FastAPI()

# Simulated database session
class DatabaseSession:
    def __init__(self):
        self.connected = False
        self.transaction_active = False
    
    def connect(self):
        if not self.connected:
            print("🔌 Opening database connection...")
            self.connected = True
        return self
    
    def begin_transaction(self):
        print("🔄 Beginning transaction...")
        self.transaction_active = True
    
    def commit(self):
        if self.transaction_active:
            print("✅ Committing transaction...")
            self.transaction_active = False
    
    def rollback(self):
        if self.transaction_active:
            print("❌ Rolling back transaction...")
            self.transaction_active = False
    
    def close(self):
        if self.connected:
            print("🔒 Closing database connection...")
            self.connected = False

# Dependency that manages database session lifecycle
def get_db_session() -> Generator[DatabaseSession, None, None]:
    """
    Dependency that:
    1. Creates a database session
    2. Begins a transaction
    3. Yields the session to the route
    4. Commits on success or rolls back on error
    5. Closes the connection
    """
    session = DatabaseSession().connect()
    session.begin_transaction()
    
    try:
        yield session
        session.commit()
    except Exception as e:
        session.rollback()
        raise e
    finally:
        session.close()

@app.get("/users/")
def get_users(db: DatabaseSession = Depends(get_db_session)):
    """Route that uses database session"""
    # In real app: users = db.query(User).all()
    return {"users": ["user1", "user2"], "db_status": "connected"}

@app.post("/users/")
def create_user(db: DatabaseSession = Depends(get_db_session)):
    """Route that creates a user (transaction will commit)"""
    # In real app: db.add(new_user)
    return {"message": "User created", "db_status": "transaction active"}

print("✅ Database session management example")
print("💡 Session is automatically managed: connect → use → commit/rollback → close")

In [ ]:
# Real-world SQLAlchemy Example Pattern

from fastapi import FastAPI, Depends
from sqlalchemy.orm import Session
from typing import Generator

# This is the pattern you'd use with real SQLAlchemy:
"""
from database import SessionLocal

def get_db() -> Generator[Session, None, None]:
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@app.get("/users/")
def get_users(db: Session = Depends(get_db)):
    users = db.query(User).all()
    return users
"""

print("✅ SQLAlchemy pattern example (commented)")
print("💡 This is the standard pattern for SQLAlchemy with FastAPI")

### 2. Authentication/Authorization

Dependencies are perfect for authentication and authorization checks!

In [ ]:
# Example: Authentication with JWT

from fastapi import FastAPI, Depends, HTTPException, Header
from typing import Optional
import jwt
from datetime import datetime, timedelta

app = FastAPI()

# Secret key (in real app, use environment variable)
SECRET_KEY = "your-secret-key-here"
ALGORITHM = "HS256"

# Simulated user database
USERS_DB = {
    "user123": {"id": "user123", "username": "john", "role": "user"},
    "admin456": {"id": "admin456", "username": "admin", "role": "admin"}
}

def create_token(user_id: str) -> str:
    """Create JWT token"""
    payload = {
        "user_id": user_id,
        "exp": datetime.utcnow() + timedelta(hours=24)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)

def verify_token(token: str) -> dict:
    """Verify JWT token"""
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except jwt.ExpiredSignatureError:
        raise HTTPException(status_code=401, detail="Token expired")
    except jwt.InvalidTokenError:
        raise HTTPException(status_code=401, detail="Invalid token")

# Dependency: Get current user from token
def get_current_user(authorization: Optional[str] = Header(None)):
    """Extract and verify user from JWT token"""
    if not authorization:
        raise HTTPException(status_code=401, detail="Authorization header required")
    
    # Extract token from "Bearer <token>"
    try:
        scheme, token = authorization.split()
        if scheme.lower() != "bearer":
            raise HTTPException(status_code=401, detail="Invalid authentication scheme")
    except ValueError:
        raise HTTPException(status_code=401, detail="Invalid authorization header")
    
    # Verify token
    payload = verify_token(token)
    user_id = payload.get("user_id")
    
    # Get user from database
    user = USERS_DB.get(user_id)
    if not user:
        raise HTTPException(status_code=401, detail="User not found")
    
    return user

# Dependency: Check if user is admin
def require_admin(current_user: dict = Depends(get_current_user)):
    """Require admin role"""
    if current_user.get("role") != "admin":
        raise HTTPException(status_code=403, detail="Admin access required")
    return current_user

@app.get("/profile/")
def get_profile(user: dict = Depends(get_current_user)):
    """Get current user's profile"""
    return {"profile": user}

@app.get("/admin/users/")
def list_all_users(admin: dict = Depends(require_admin)):
    """Admin-only endpoint"""
    return {"users": list(USERS_DB.values()), "accessed_by": admin["username"]}

# Helper endpoint to get a token (for testing)
@app.post("/login/")
def login(username: str):
    """Login and get token (simplified)"""
    # Find user
    user = next((u for u in USERS_DB.values() if u["username"] == username), None)
    if not user:
        raise HTTPException(status_code=404, detail="User not found")
    
    token = create_token(user["id"])
    return {"access_token": token, "token_type": "bearer"}

print("✅ Authentication/Authorization example")
print("💡 Use /login/ to get a token, then use it in Authorization header")

In [ ]:
# Example: Role-based Access Control (RBAC)

from fastapi import FastAPI, Depends, HTTPException
from typing import List

app = FastAPI()

# Simulated current user (in real app, from JWT)
CURRENT_USER = {"id": "user123", "username": "john", "roles": ["user", "editor"]}

def get_current_user():
    """Get current user"""
    return CURRENT_USER

def require_role(required_role: str):
    """Factory function that creates a dependency requiring a specific role"""
    def role_checker(user: dict = Depends(get_current_user)):
        user_roles = user.get("roles", [])
        if required_role not in user_roles:
            raise HTTPException(
                status_code=403,
                detail=f"Role '{required_role}' required. Your roles: {user_roles}"
            )
        return user
    return role_checker

# Create dependencies for different roles
require_editor = require_role("editor")
require_admin = require_role("admin")

@app.get("/posts/")
def list_posts(user: dict = Depends(get_current_user)):
    """Anyone can list posts"""
    return {"posts": ["post1", "post2"], "user": user["username"]}

@app.post("/posts/")
def create_post(user: dict = Depends(require_editor)):
    """Only editors can create posts"""
    return {"message": "Post created", "created_by": user["username"]}

@app.delete("/posts/{post_id}")
def delete_post(post_id: int, user: dict = Depends(require_admin)):
    """Only admins can delete posts"""
    return {"message": f"Post {post_id} deleted", "deleted_by": user["username"]}

print("✅ Role-based access control example")
print("💡 Different endpoints require different roles")

### 3. Request Validation

Use dependencies to validate and transform request data.

In [ ]:
# Example: Request Validation with Dependencies

from fastapi import FastAPI, Depends, Query, HTTPException
from typing import Optional
from datetime import datetime

app = FastAPI()

# Validation dependency for date range
def validate_date_range(
    start_date: Optional[str] = Query(None),
    end_date: Optional[str] = Query(None)
):
    """Validate and parse date range"""
    if start_date and end_date:
        try:
            start = datetime.fromisoformat(start_date)
            end = datetime.fromisoformat(end_date)
            
            if start > end:
                raise HTTPException(
                    status_code=400,
                    detail="start_date must be before end_date"
                )
            
            # Check date range is not too large (e.g., max 1 year)
            if (end - start).days > 365:
                raise HTTPException(
                    status_code=400,
                    detail="Date range cannot exceed 365 days"
                )
            
            return {"start_date": start, "end_date": end}
        except ValueError:
            raise HTTPException(status_code=400, detail="Invalid date format (use ISO format)")
    
    return {"start_date": None, "end_date": None}

# Validation dependency for pagination
def validate_pagination(
    page: int = Query(1, ge=1, description="Page number"),
    page_size: int = Query(10, ge=1, le=100, description="Items per page")
):
    """Validate pagination parameters"""
    return {
        "page": page,
        "page_size": page_size,
        "skip": (page - 1) * page_size,
        "limit": page_size
    }

@app.get("/reports/")
def get_reports(
    date_range: dict = Depends(validate_date_range),
    pagination: dict = Depends(validate_pagination)
):
    """Get reports with validated date range and pagination"""
    return {
        "date_range": date_range,
        "pagination": pagination,
        "reports": [f"report_{i}" for i in range(pagination["skip"], pagination["skip"] + pagination["limit"])]
    }

print("✅ Request validation example")
print("💡 Dependencies validate and transform request parameters")

In [ ]:
# Example: Complex Validation with Pydantic Models

from fastapi import FastAPI, Depends, Query
from pydantic import BaseModel, Field, validator
from typing import List, Optional

app = FastAPI()

class FilterParams(BaseModel):
    """Pydantic model for filter parameters"""
    categories: Optional[List[str]] = Field(None, description="Filter by categories")
    min_price: Optional[float] = Field(None, ge=0, description="Minimum price")
    max_price: Optional[float] = Field(None, ge=0, description="Maximum price")
    tags: Optional[List[str]] = Field(None, description="Filter by tags")
    
    @validator('max_price')
    def validate_price_range(cls, v, values):
        """Validate that max_price > min_price"""
        if v and 'min_price' in values and values['min_price']:
            if v < values['min_price']:
                raise ValueError('max_price must be greater than min_price')
        return v

def get_filters(
    categories: Optional[str] = Query(None, description="Comma-separated categories"),
    min_price: Optional[float] = Query(None, ge=0),
    max_price: Optional[float] = Query(None, ge=0),
    tags: Optional[str] = Query(None, description="Comma-separated tags")
) -> FilterParams:
    """Parse and validate filter parameters"""
    return FilterParams(
        categories=categories.split(",") if categories else None,
        min_price=min_price,
        max_price=max_price,
        tags=tags.split(",") if tags else None
    )

@app.get("/products/")
def list_products(filters: FilterParams = Depends(get_filters)):
    """List products with validated filters"""
    return {
        "filters": filters.dict(exclude_none=True),
        "products": ["product1", "product2", "product3"]
    }

print("✅ Complex validation with Pydantic example")

### 4. Rate Limiting

Use dependencies to implement rate limiting per user or IP.

In [ ]:
# Example: Rate Limiting with Dependencies

from fastapi import FastAPI, Depends, HTTPException, Request
from collections import defaultdict
from datetime import datetime, timedelta
from typing import Dict

app = FastAPI()

# In-memory rate limit storage (use Redis in production)
rate_limit_store: Dict[str, list] = defaultdict(list)

def rate_limit(max_requests: int = 10, window_seconds: int = 60):
    """
    Factory function that creates a rate limiting dependency
    
    Args:
        max_requests: Maximum number of requests allowed
        window_seconds: Time window in seconds
    """
    def check_rate_limit(request: Request):
        """Check if request is within rate limit"""
        # Get client identifier (IP address)
        client_ip = request.client.host
        
        # Get current time
        now = datetime.utcnow()
        
        # Clean old requests outside the window
        window_start = now - timedelta(seconds=window_seconds)
        rate_limit_store[client_ip] = [
            req_time for req_time in rate_limit_store[client_ip]
            if req_time > window_start
        ]
        
        # Check if limit exceeded
        if len(rate_limit_store[client_ip]) >= max_requests:
            raise HTTPException(
                status_code=429,
                detail=f"Rate limit exceeded: {max_requests} requests per {window_seconds} seconds"
            )
        
        # Record this request
        rate_limit_store[client_ip].append(now)
        
        return {
            "remaining": max_requests - len(rate_limit_store[client_ip]),
            "reset_in": window_seconds
        }
    
    return check_rate_limit

# Create rate limiters with different limits
strict_rate_limit = rate_limit(max_requests=5, window_seconds=60)  # 5 requests per minute
normal_rate_limit = rate_limit(max_requests=100, window_seconds=60)  # 100 requests per minute

@app.get("/api/strict/")
def strict_endpoint(rate_limit_info: dict = Depends(strict_rate_limit)):
    """Endpoint with strict rate limiting (5 req/min)"""
    return {
        "message": "Success",
        "rate_limit": rate_limit_info
    }

@app.get("/api/normal/")
def normal_endpoint(rate_limit_info: dict = Depends(normal_rate_limit)):
    """Endpoint with normal rate limiting (100 req/min)"""
    return {
        "message": "Success",
        "rate_limit": rate_limit_info
    }

print("✅ Rate limiting example")
print("💡 Different endpoints can have different rate limits")

In [ ]:
# Example: User-based Rate Limiting

from fastapi import FastAPI, Depends, HTTPException, Header
from collections import defaultdict
from datetime import datetime, timedelta
from typing import Dict, Optional

app = FastAPI()

# Rate limit storage per user
user_rate_limits: Dict[str, list] = defaultdict(list)

def get_user_id(authorization: Optional[str] = Header(None)) -> str:
    """Extract user ID from authorization header"""
    if authorization:
        # In real app, decode JWT to get user_id
        return authorization.split(":")[-1] if ":" in authorization else "user_123"
    return "anonymous"

def user_rate_limit(max_requests: int = 20, window_seconds: int = 60):
    """Rate limit based on user ID"""
    def check_user_rate_limit(
        user_id: str = Depends(get_user_id)
    ):
        now = datetime.utcnow()
        window_start = now - timedelta(seconds=window_seconds)
        
        # Clean old requests
        user_rate_limits[user_id] = [
            req_time for req_time in user_rate_limits[user_id]
            if req_time > window_start
        ]
        
        # Check limit
        if len(user_rate_limits[user_id]) >= max_requests:
            raise HTTPException(
                status_code=429,
                detail=f"Rate limit exceeded for user. {max_requests} requests per {window_seconds} seconds"
            )
        
        # Record request
        user_rate_limits[user_id].append(now)
        
        return {
            "user_id": user_id,
            "remaining": max_requests - len(user_rate_limits[user_id]),
            "reset_in": window_seconds
        }
    
    return check_user_rate_limit

@app.get("/user-api/")
def user_endpoint(rate_info: dict = Depends(user_rate_limit(max_requests=10, window_seconds=60))):
    """Endpoint with user-based rate limiting"""
    return {
        "message": "Success",
        "rate_limit": rate_info
    }

print("✅ User-based rate limiting example")

### 5. Logging and Monitoring

Use dependencies to add logging and monitoring to your endpoints.

In [ ]:
# Example: Logging Dependency

from fastapi import FastAPI, Depends, Request
from datetime import datetime
import logging

app = FastAPI()

# Setup logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def log_request(request: Request):
    """Dependency that logs request information"""
    start_time = datetime.utcnow()
    
    log_data = {
        "method": request.method,
        "url": str(request.url),
        "path": request.url.path,
        "client_ip": request.client.host,
        "user_agent": request.headers.get("user-agent"),
        "timestamp": start_time.isoformat()
    }
    
    logger.info(f"Request: {log_data}")
    
    return {
        "start_time": start_time,
        "log_data": log_data
    }

@app.get("/logged-endpoint/")
def logged_endpoint(log_info: dict = Depends(log_request)):
    """Endpoint with request logging"""
    return {
        "message": "Success",
        "logged_at": log_info["log_data"]["timestamp"]
    }

print("✅ Logging dependency example")

In [ ]:
# Example: Performance Monitoring Dependency

from fastapi import FastAPI, Depends, Request
from datetime import datetime
import time
from typing import Dict

app = FastAPI()

# Metrics storage (use proper metrics service in production)
metrics_store = {
    "request_count": 0,
    "total_duration": 0.0,
    "endpoints": {}
}

def monitor_performance(request: Request):
    """Dependency that monitors endpoint performance"""
    start_time = time.time()
    endpoint = request.url.path
    
    def record_metrics():
        """Record metrics after request completes"""
        duration = time.time() - start_time
        
        metrics_store["request_count"] += 1
        metrics_store["total_duration"] += duration
        
        if endpoint not in metrics_store["endpoints"]:
            metrics_store["endpoints"][endpoint] = {
                "count": 0,
                "total_duration": 0.0,
                "avg_duration": 0.0
            }
        
        metrics_store["endpoints"][endpoint]["count"] += 1
        metrics_store["endpoints"][endpoint]["total_duration"] += duration
        metrics_store["endpoints"][endpoint]["avg_duration"] = (
            metrics_store["endpoints"][endpoint]["total_duration"] /
            metrics_store["endpoints"][endpoint]["count"]
        )
        
        return {
            "duration": duration,
            "endpoint": endpoint,
            "metrics": metrics_store["endpoints"][endpoint]
        }
    
    return record_metrics

@app.get("/monitored-endpoint/")
def monitored_endpoint(record: callable = Depends(monitor_performance)):
    """Endpoint with performance monitoring"""
    # Simulate some work
    time.sleep(0.1)
    
    metrics = record()  # Record metrics
    
    return {
        "message": "Success",
        "performance": {
            "duration_ms": round(metrics["duration"] * 1000, 2),
            "avg_duration_ms": round(metrics["metrics"]["avg_duration"] * 1000, 2)
        }
    }

@app.get("/metrics/")
def get_metrics():
    """Get application metrics"""
    return {
        "total_requests": metrics_store["request_count"],
        "avg_duration": (
            metrics_store["total_duration"] / metrics_store["request_count"]
            if metrics_store["request_count"] > 0 else 0
        ),
        "endpoints": metrics_store["endpoints"]
    }

print("✅ Performance monitoring example")
print("💡 Call /monitored-endpoint/ multiple times, then check /metrics/")

In [ ]:
# Example: Comprehensive Logging and Monitoring

from fastapi import FastAPI, Depends, Request
from datetime import datetime
import time
import logging
from typing import Dict

app = FastAPI()

# Setup structured logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class RequestLogger:
    """Class-based dependency for comprehensive request logging"""
    
    def __init__(self, request: Request):
        self.request = request
        self.start_time = time.time()
        self.request_id = f"req-{int(time.time() * 1000)}"
        
    def log_start(self):
        """Log request start"""
        logger.info(
            f"[{self.request_id}] {self.request.method} {self.request.url.path} - "
            f"IP: {self.request.client.host} - Started"
        )
        return self
    
    def log_success(self, response_data: dict):
        """Log successful request"""
        duration = time.time() - self.start_time
        logger.info(
            f"[{self.request_id}] {self.request.method} {self.request.url.path} - "
            f"Success - Duration: {duration:.3f}s"
        )
        return {"request_id": self.request_id, "duration": duration}
    
    def log_error(self, error: Exception):
        """Log error"""
        duration = time.time() - self.start_time
        logger.error(
            f"[{self.request_id}] {self.request.method} {self.request.url.path} - "
            f"Error: {str(error)} - Duration: {duration:.3f}s"
        )
        raise error

def get_request_logger(request: Request) -> RequestLogger:
    """Dependency that provides request logger"""
    logger = RequestLogger(request)
    return logger.log_start()

@app.get("/comprehensive-logging/")
def comprehensive_endpoint(
    logger: RequestLogger = Depends(get_request_logger)
):
    """Endpoint with comprehensive logging"""
    try:
        # Simulate work
        time.sleep(0.05)
        
        result = {"message": "Success", "data": "some data"}
        log_info = logger.log_success(result)
        
        return {**result, "request_id": log_info["request_id"]}
    except Exception as e:
        logger.log_error(e)

print("✅ Comprehensive logging example")

## Summary: Key Takeaways 🎯

### What We Learned:

1. **Dependency Injection Basics**
   - Dependencies are functions/classes that FastAPI automatically calls
   - Use `Depends()` to inject dependencies into routes
   - Dependencies can be functions, classes, or async functions

2. **Dependency Scopes**
   - Request scope (default): Created for each request
   - Cached: Use `@lru_cache` for expensive operations
   - Application scope: Use startup/shutdown events

3. **Advanced Patterns**
   - Class-based dependencies for complex state
   - Sub-dependencies (dependencies of dependencies)
   - Dependency overrides for testing
   - Caching with `@lru_cache`

4. **Common Use Cases**
   - Database session management
   - Authentication/Authorization
   - Request validation
   - Rate limiting
   - Logging and monitoring

### Best Practices:

✅ **Use dependencies for reusable logic** (auth, validation, etc.)
✅ **Keep dependencies focused** (single responsibility)
✅ **Use dependency overrides for testing**
✅ **Cache expensive dependencies** with `@lru_cache`
✅ **Use generators for cleanup** (database sessions, file handles)
✅ **Document dependencies** in docstrings

### Next Steps:

- Practice building real applications with dependencies
- Explore FastAPI's official documentation
- Try combining multiple dependency patterns
- Build your own custom dependencies for your use cases

Happy coding! 🚀